## 5.1 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)

net(X)

tensor([[-0.1268, -0.0102, -0.0102,  0.0718,  0.0312, -0.1643, -0.1308, -0.0702,
          0.0203,  0.0316],
        [-0.0340, -0.0225,  0.0436,  0.1739, -0.0230, -0.0728, -0.2050,  0.0085,
         -0.0846,  0.0119]], grad_fn=<AddmmBackward0>)

### 5.1.1 自定义块

In [3]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.out = nn.Linear(256, 10)
  
  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))


In [4]:
net = MLP()
net(X)

tensor([[-0.0849,  0.2624, -0.2501, -0.0997, -0.0120, -0.0288, -0.0548, -0.0665,
          0.1897, -0.3211],
        [-0.1591,  0.2209, -0.3900, -0.1512, -0.1119,  0.0107, -0.1270, -0.0778,
          0.1561, -0.2947]], grad_fn=<AddmmBackward0>)

### 5.1.2 顺序快

In [5]:
# 继承了nn.Module
class MySequential(nn.Module):
  # 初始化
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      self._modules[str(idx)] = module

  # 前向传播
  def forward(self, X):
    for block in self._modules.values():
      X = block(X)
    return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1580,  0.0936, -0.0525, -0.1129,  0.1969,  0.0965, -0.2938,  0.0179,
         -0.2397,  0.1776],
        [ 0.2329,  0.0508,  0.0281, -0.0890,  0.1963,  0.1339, -0.2175, -0.0566,
         -0.1834,  0.1582]], grad_fn=<AddmmBackward0>)

### 5.1.3 在前向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    # 不计算梯度的随机权重参数，因此在训练期间保持不变。
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.linear = nn.Linear(20, 20)

  """
  -> relu里面为什么+1？
  具体来说，这里的+1操作是为了确保ReLU函数之前的输入torch.mm(X, self.rand_weight) + 1至少有一部分是正数，
  这可以保证在应用ReLU激活函数时有非零的输出。提高了非线性的表达能力，确保数值的稳定性。
  """
  def forward(self, X):
    X = self.linear(X)
    # 使用是创建的敞亮参数和relu和mm函数
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    # 复用了第一层参数。
    X = self.linear(X)
    """
    下述代码可能不会常用于在任何实际任务中
    只是展示如何将任意代码集成到神经网络计算的流程中。
    """
    while X.abs().sum() > 1:
      X /= 2
    return X.sum(0)

In [15]:
net = FixedHiddenMLP()
net(X)

tensor([ 0.0655, -0.0216, -0.0316, -0.0350,  0.0532, -0.0492,  0.1080,  0.0104,
         0.0175,  0.0419,  0.0433,  0.0069, -0.0242, -0.0723, -0.0152, -0.0176,
        -0.0330,  0.0271,  0.0314,  0.0235], grad_fn=<SumBackward1>)

In [16]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                             nn.Linear(64, 32), nn.ReLU())
    self.linear = nn.Linear(32, 16)
  
  def forward(self, X):
    return self.linear(self.net(X))

In [17]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor([ 0.0340,  0.0290,  0.0231,  0.0500, -0.0071,  0.0160, -0.0217,  0.0210,
        -0.0187, -0.0058, -0.0111, -0.0554, -0.0167, -0.0302, -0.0227,  0.0135,
         0.0284, -0.0244,  0.0759, -0.0155], grad_fn=<SumBackward1>)

### 练习

1. 如果将MySequential中存储块的方式更改为Python列表，会出现什么样的问题？

(1). 参数注册问题。在 PyTorch 中，模型的所有参数（如权重和偏置）需要在模型的 parameters() 方法中正确注册，以便在模型训练时这些参数可以被识别和更新。
(2). 设备兼容性问题。当你将模型从一个设备（如 CPU）迁移到另一个设备（如 GPU）时，使用 nn.Module 的方法（如 .to(device)）可以自动处理所有注册的子模块的设备迁移。
(3). 使用 PyTorch 的模型保存和加载功能（如 torch.save 和 torch.load）时，模型的状态包括其所有注册的参数会被保存。如果层存储在普通列表中，它们可能不会被正确地保存或加载。

2. 实现一个块，它以两个块为参数，例如net1和net2，并返回前向传播中两个网络的串联输出。这也被称为平行块。

In [19]:
# 2. 实现平行块

class MyNet1(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(20, 256),
      nn.ReLU(),
      nn.Linear(256, 40)
    )
  
  def forward(self, X):
    return self.net(X)

class MyNet2(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(40, 20),
      nn.ReLU(),
      nn.Linear(20, 10)
    )
  
  def forward(self, X):
    return self.net(X)
  
mynet = nn.Sequential(MyNet1(), MyNet2())
mynet(X)

tensor([[ 0.2088, -0.0857,  0.3436, -0.0044,  0.2110,  0.0057,  0.1934, -0.0516,
          0.0959, -0.2448],
        [ 0.2062, -0.0670,  0.3424, -0.0007,  0.2077, -0.0176,  0.1722, -0.0668,
          0.0930, -0.2480]], grad_fn=<AddmmBackward0>)

3. 暂时不写了。

## 5.2 参数管理

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3530],
        [-0.3241]], grad_fn=<AddmmBackward0>)

### 5.2.1 参数访问

In [10]:
# 以字典形式返回该层所有参数。

print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2554,  0.3526, -0.0392, -0.3447, -0.1620, -0.1505,  0.1045,  0.0571]])), ('bias', tensor([-0.2548]))])


In [11]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2548], requires_grad=True)
tensor([-0.2548])


In [13]:
# 返回所有参数

print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
# 另一种访问网络参数的方式

net.state_dict()['2.bias'].data

tensor([-0.2548])

In [25]:
def block1():
  return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
         nn.Linear(8, 4), nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}', block1())
  return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1707],
        [0.1705]], grad_fn=<AddmmBackward0>)

In [26]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [27]:
# 第一个主要块、第二个子块、第一层的偏置项

rgnet[0][1][0].bias.data

tensor([-0.1695,  0.4668,  0.1065,  0.1952, -0.2350,  0.2330,  0.0304, -0.2151])

### 5.2.2 参数初始化

In [17]:
# 内置初始化方式
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)

# 应用初始化方式
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0135,  0.0136, -0.0074,  0.0178]), tensor(0.))

In [31]:
# 常量初始化方式
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 1)
    nn.init.zeros_(m.bias)

# 应用初始化方式
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [32]:
def init_xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)

def init_42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].apply(init_xavier))
print(net[2].apply(init_42))

Linear(in_features=4, out_features=8, bias=True)
Linear(in_features=8, out_features=1, bias=True)


In [33]:
# 实现自定义的初始化方式

def my_init(m):
  if type(m) == nn.Linear:
    print("Init", *[(name, param.shape) 
                    for name, param in m.named_parameters()][0])
    
    nn.init.uniform_(m.weight, -10, 10)
    m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)

net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.5857, -0.0000,  0.0000,  8.0813],
        [-8.0285, -0.0000, -7.4692, -6.2517]], grad_fn=<SliceBackward0>)

In [34]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  9.0813])

### 5.2.3 参数绑定

In [18]:
# 这个例子表明第三个和第五个神经网络层的参数是绑定的。 它们不仅值相等，而且由相同的张量表示。

shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)

print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 练习

1. 使用 5.1节 中定义的FancyMLP模型，访问各个层的参数。

没看到这个模型。

In [21]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                             nn.Linear(64, 32), nn.ReLU())
    self.linear = nn.Linear(32, 16)
  
  def forward(self, X):
    return self.linear(self.net(X))

In [23]:
net = NestMLP()
net

NestMLP(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (linear): Linear(in_features=32, out_features=16, bias=True)
)

In [24]:
# 访问所有参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('net.0.weight', torch.Size([64, 20])) ('net.0.bias', torch.Size([64])) ('net.2.weight', torch.Size([32, 64])) ('net.2.bias', torch.Size([32])) ('linear.weight', torch.Size([16, 32])) ('linear.bias', torch.Size([16]))


In [33]:
print(*[(name, param.shape) for name, param in net.net.named_parameters()])

('0.weight', torch.Size([64, 20])) ('0.bias', torch.Size([64])) ('2.weight', torch.Size([32, 64])) ('2.bias', torch.Size([32]))


In [29]:
net.net

Sequential(
  (0): Linear(in_features=20, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
)

In [32]:
net.net[0], net.net[0].weight, net.net[0].bias

(Linear(in_features=20, out_features=64, bias=True),
 Parameter containing:
 tensor([[-1.2013e-01, -8.4290e-03,  7.5989e-02,  ..., -1.5526e-01,
           1.7419e-02, -1.2501e-01],
         [-1.7252e-01,  6.3889e-03, -1.7741e-01,  ..., -4.9994e-02,
           8.8070e-02,  2.1124e-01],
         [-1.9235e-01, -2.0185e-01, -5.8160e-02,  ...,  1.1672e-01,
          -1.3069e-01, -7.1507e-02],
         ...,
         [ 2.1606e-01,  1.9329e-01,  8.3897e-02,  ..., -2.3049e-02,
           1.0789e-02,  2.0129e-01],
         [-1.8019e-01, -4.8555e-02,  1.2082e-01,  ..., -8.1708e-02,
          -2.1021e-01,  1.8771e-01],
         [-1.1916e-01, -1.0058e-02, -1.9220e-04,  ...,  6.2949e-02,
           1.8632e-01, -1.4527e-01]], requires_grad=True),
 Parameter containing:
 tensor([-0.1135, -0.0099,  0.1866,  0.1538,  0.1854, -0.0577,  0.0538, -0.0680,
          0.0938,  0.0094, -0.0815,  0.0848, -0.0556, -0.0611, -0.2111,  0.0473,
         -0.0614, -0.1494, -0.0977, -0.0953,  0.0163,  0.1950, -0.0971,  

2. 查看初始化模块文档以了解不同的初始化方法。

看完了

3. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。

In [34]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

4. 为什么共享参数是个好主意？


1. 减少模型的参数数量：这可以减轻过拟合的风险，尤其是在训练数据较少的情况下。

2. 提高训练效率：由于参数数量减少，模型训练所需的计算资源也相应减少，从而可能加快训练过程。

3. 增强模型的泛化能力：共享权重迫使网络在多个位置学习相似的特征，这可能有助于模型在处理类似但位置不同的特征时表现得更好。

对于卷积神经网络（CNN），参数共享最为显著的体现是通过卷积核（或过滤器）实现的。卷积核在输入数据（如图像）的整个区域滑动，并在每个位置应用相同的权重集合。这些特点使得参数共享在CNN中具有以下优势：

- 降低过拟合：由于使用的参数数量减少，模型更不容易过拟合于训练数据。
- 减少参数数量：相比于全连接层，卷积层由于其权重共享特性，可以显著减少需要学习的参数数量。
- 提高模型的空间不变性：通过在图像的不同位置应用相同的卷积核，模型可以识别无论位置在哪里的相同特征，增强了模型对于输入数据平移的不变性。

对于循环神经网络（RNN），在RNN中，参数共享表现为在不同时间步骤中使用相同的权重。这种网络结构特别适用于处理序列数据，如文本或时间序列数据。参数共享使得RNN具有以下特性：

- 处理任意长度的序列：由于在每个时间步使用相同的参数，RNN能够处理任意长度的输入序列。
- 学习序列中的长期依赖关系：相同的权重在整个序列中反复应用，帮助网络捕捉序列内部的时间动态和长距离的依赖关系。


## 5.3 延后初始化（Pytorch暂未更新）

## 5.4 自定义层

### 5.4.1 不带参数的层

In [37]:
import torch
import torch.nn.functional as F
from torch import nn

In [40]:
class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, X):
    return X - X.mean()

In [41]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [42]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [43]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.7789e-09, grad_fn=<MeanBackward0>)

### 5.4.2 带参数的层

In [44]:
class MyLinear(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(in_units, units))
    self.bias = nn.Parameter(torch.randn(units,))
  def forward(self, X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)


In [45]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.3761, -1.4402,  0.3196],
        [ 0.1689,  1.4698,  0.7660],
        [ 0.2874,  0.2391, -0.7497],
        [-0.2998,  0.0715, -1.4424],
        [-1.0007, -2.6984,  1.1358]], requires_grad=True)

In [46]:
linear(torch.rand(2, 5))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [47]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))

net(torch.rand(2, 64))

tensor([[0.0000],
        [0.6949]])

### 练习

1. 张量降维层

In [47]:
class QuadraticLayer(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(output_dim, input_dim, input_dim))
  
  def forward(self, x):
    # 在这里实现具体的代码，不想写了，返回个0吧。。。
    return 0

In [48]:
# 示例使用
input_dim = 4
output_dim = 3
batch_size = 2
x = torch.randn(batch_size, input_dim)  # 随机生成输入数据
layer = nn.Sequential(QuadraticLayer(input_dim, output_dim))
output = layer(x)

print("Input:")
print(x)
print("Output:")
print(output)

Input:
tensor([[-0.9123,  0.0247,  2.0235,  0.7034],
        [-0.2116, -0.0587, -0.6523,  0.8740]])
Output:
0


2. 一个返回输入数据的傅立叶系数前半部分的层。

和上一问差不多，改一下forward就行。

## 5.5 读写文件

### 5.5.1 加载和保存张量

In [48]:
import torch
from torch import nn
from torch.nn import functional as F

In [49]:
x = torch.arange(4)

torch.save(x, 'x-file')

In [51]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [52]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')

(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [53]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')

mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.5.2 加载和保存模型参数

In [54]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.output = nn.Linear(256, 10)

  def forward(self, x):
    return self.output(F.relu(self.hidden(x)))

In [55]:
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [56]:
torch.save(net.state_dict(), 'mlp.params')

In [57]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [58]:
Y_clone = clone(X)
Y_clone = Y

### 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？（ChatGPT）

即使在不考虑跨设备部署的情况下，存储经过训练的模型参数还是具有多种实际好处的。这些好处不仅对模型的开发和迭代过程至关重要，还有助于模型的验证、复现和进一步的分析。以下是一些存储模型参数的主要好处：

(1). 模型验证和测试
- 结果复现：存储训练后的模型参数使得可以在不同时间点准确复现模型的性能，这对科学研究和实验验证非常重要。
- 性能评估：可以在新的或未见过的数据上重复使用相同的模型进行评估，以验证模型的泛化能力。

(2). 模型迭代和改进
- 继续训练：在现有的训练基础上继续训练模型（如在新数据上进行微调）时，直接从保存的参数开始可以节省大量时间。
- 参数调整：通过分析保存的模型参数，开发者可以对模型架构做出更有针对性的调整，如修改那些权重几乎没有变化的层。

(3). 团队协作和传递
- 团队共享：模型参数的存储使得团队成员之间可以轻松共享同一个模型的状态，便于协作和进一步开发。
- 成果展示：在向利益相关者展示模型成果时，可以随时加载预训练模型来演示其性能。

(4). 模型备份
- 避免丢失：在训练过程中，尤其是长时间的训练，存在硬件故障或其他中断训练的风险。保存参数可作为备份，以防意外发生。

(5). 商业部署和产品开发
- 产品集成：商业产品可以直接集成预训练模型来提供智能功能，如推荐系统、自动分类等。
- 许可和分销：对于软件供应商，训练好的模型可以作为产品的一部分进行许可或分销。

(6). 科研和出版
- 研究出版：在科研领域，公开模型的参数是确保其他研究人员可以验证和复现您的研究结果的重要步骤。
- 开源贡献：将模型参数开源可以促进技术的共享和行业的快速发展。

综上所述。存储模型参数具有广泛的应用价值，它不仅可以帮助确保模型性能的一致性，还能在模型开发、协作和应用方面提供重要支持。因此，即便是在单一设备或环境中使用模型，合理管理和存储模型参数也是模型开发和维护中的一个重要方面。

2. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

In [ ]:
# =================STEP 1. 定义原始网络=================

class OriginalNet(nn.Module):
    def __init__(self):
        super(OriginalNet, self).__init__()
        self.layer1 = nn.Linear(10, 20)
        self.layer2 = nn.Linear(20, 30)
        self.layer3 = nn.Linear(30, 40)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 实例化并加载预训练权重（假设模型已经训练）
original_net = OriginalNet()
# original_net.load_state_dict(torch.load('path_to_trained_weights.pth'))

# =================STEP 2. 定义新的网络=================

class NewNet(nn.Module):
    def __init__(self, original_net):
        super(NewNet, self).__init__()
        self.layer1 = original_net.layer1
        self.layer2 = original_net.layer2
        self.new_layer = nn.Linear(30, 50)  # 新添加的层

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.new_layer(x)
        return x

# 使用原始网络的前两层来创建新网络
new_net = NewNet(original_net)

# =================STEP 3. 冻结前两层=================

# 冻结前两层
for param in new_net.layer1.parameters():
    param.requires_grad = False
for param in new_net.layer2.parameters():
    param.requires_grad = False

# 接下来，你可以按照常规方式训练新网络



- 当你从一个模型中提取层到另一个模型时，需要确保数据的维度匹配。

- 冻结参数是为了避免在新任务上训练时修改预训练层的权重，这在处理类似但不完全相同的任务时特别有用。

3. 如何同时保存网络架构和参数？需要对架构加上什么限制？

需要注意的事项和架构限制

- 代码一致性：在加载模型时，必须保证模型类的定义（如MyModel）与保存时完全相同。这意味着加载模型的环境中必须有相同的类定义，包括所有层的初始化和forward方法的定义。

- 自定义层和函数：如果模型中使用了自定义层或函数，这些定义也需要在加载模型的环境中可用。如果这些自定义组件涉及复杂的Python逻辑或外部库依赖，可能会导致模型无法直接加载。

- 设备一致性：在保存和加载模型时，应注意模型可能在不同的设备（CPU或GPU）之间移动。在加载模型后，可能需要将模型转移到适当的设备上。

- 安全考虑：使用torch.load时要注意，不要加载不可信的文件，因为这可能导致安全风险。如果可能，使用torch.load的pickle_module和pickle_load_args参数来限制可加载对象的范围。

## 5.6 GPU

In [59]:
!nvidia-smi

Sun Apr 28 14:13:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.61                 Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050       WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   39C    P8               N/A /  N/A|    131MiB /  3072MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 5.6.1 计算设备

In [4]:
import torch
from torch import nn

In [5]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [6]:
torch.cuda.device_count()

1

In [60]:
def try_gpu(i=0): #@save
  if torch.cuda.device_count() >= i + 1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus(): #@save
  devices = [torch.device(f'cuda:{i}')
              for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]


try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### 5.6.2 张量与GPU

In [8]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [9]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [16]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.4019, 0.8708, 0.3345],
        [0.9365, 0.7922, 0.1155]], device='cuda:0')

In [17]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [18]:
Y + Z

tensor([[1.4019, 1.8708, 1.3345],
        [1.9365, 1.7922, 1.1155]], device='cuda:0')

In [20]:
Z.cuda(0) is Z

True

### 5.6.3 神经网络与GPU

In [ ]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [ ]:
net(X)

In [ ]:
net[0].weight.data.device

### 练习

1. 尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？

In [51]:
from d2l import torch as d2l

In [69]:
time = d2l.Timer()

X = torch.randn((5000, 5000))
Y = torch.randn((5000, 5000))

time.start()
torch.mm(X, Y)
time.stop()

0.9689760208129883

In [71]:
X = torch.randn((10000, 10000), device=try_gpu(0))
Y = torch.randn((10000, 10000), device=try_gpu(0))

time.start()
torch.mm(X, Y)
time.stop()

0.003335714340209961

计算量大的任务GPU的能力显著超过CPU

In [78]:
time = d2l.Timer()

X = torch.randn((50, 50))
Y = torch.randn((50, 50))

time.start()
torch.mm(X, Y)
time.stop()

0.0

In [74]:
X = torch.randn((50, 50), device=try_gpu(0))
Y = torch.randn((50, 50), device=try_gpu(0))

time.start()
torch.mm(X, Y)
time.stop()

0.0

计算量小的任务，两者差不多。

2. 我们应该如何在GPU上读写模型参数？

In [ ]:
# =============STEP 1.训练模型并保存参数===============

import torch
import torch.nn as nn

# 定义模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer = nn.Linear(10, 5)

    def forward(self, x):
        return self.layer(x)

# 实例化模型并移动到GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyModel().to(device)

# 假设输入和目标数据
input = torch.randn(10, 10).to(device)  # 随机数据也要放到GPU
target = torch.randn(10, 5).to(device)  # 同上

# 训练模型
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

model.train()  # 设置为训练模式
for epoch in range(10):
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

# 保存模型参数
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
# =============STEP 2.加载模型参数===============


# 加载模型参数到指定设备
model = MyModel()  # 新实例化一个模型
model.load_state_dict(torch.load('model_weights.pth', map_location=device))

# 如果你不确定下一步操作将在哪个设备上执行，可以这样动态指定设备
model.to(device)

3. 测量计算1000个 100*100 矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。

In [9]:
import torch
from d2l import torch as d2l


timer = d2l.Timer()


timer.start()
for i in range(1000):
  # 创建一个矩阵
  A = torch.randn((100, 100))
  B = torch.randn((100, 100))
  C = torch.mm(A, B)
  torch.linalg.norm(C, 'fro')
timer.stop()

0.18849658966064453

4. 测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。

没有两个GPU。PASS